In [1]:
import numpy as np
import pymc3

/Users/stephenhky/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# radioactive element
# parameter
nbatoms = 10000
tau = 234.
atomstates = np.array([1]*nbatoms, dtype=np.int32)   # 1: original; 0: decayed

T = 500
N = [np.sum(atomstates)]*T
tseries = np.arange(T)
for t in range(1, T):
    for i in range(nbatoms):
        if atomstates[i] == 1:
            if np.random.uniform() < 1/tau:
                atomstates[i] = 0
    N[t] = np.sum(atomstates)            

In [3]:
from pymc3 import Model, Exponential, Gamma

# Defining the model
radioactive_model = Model()

with radioactive_model:
    lamb = Gamma('lamb', alpha=0.6*nbatoms, beta=nbatoms)
    pred_N = Exponential('pred_N', -lamb, observed=N)

In [4]:
from pymc3 import find_MAP

map_estimate = find_MAP(model=radioactive_model)

print(map_estimate)

/Users/stephenhky/anaconda/lib/python2.7/site-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = -inf, ||grad|| = 1.0002: 100%|██████████| 4/4 [00:00<00:00, 593.21it/s]


{'lamb': array(0.59984181), 'lamb_log__': array(-0.51108931)}


In [5]:
from pymc3 import find_MAP, NUTS, sample
from scipy.optimize import fmin_powell

with radioactive_model:
    start = find_MAP(fmin=fmin_powell)
    
    step = NUTS(scaling=start)
    
    trace = sample(2000, step, start=start)

print(trace)

/Users/stephenhky/anaconda/lib/python2.7/site-packages/pymc3/tuning/starting.py:102: UserWarning: In future versions, set the optimization algorithm with a string. For example, use `method="L-BFGS-B"` instead of `fmin=sp.optimize.fmin_l_bfgs_b"`.
  warnings.warn('In future versions, set the optimization algorithm with a string. '
logp = -inf:   0%|          | 13/5000 [00:00<00:14, 351.59it/s] 


Optimization terminated successfully.
         Current function value: -10000000000000000159028911097599180468360808563945281389781327557747838772170381060813469985856815104.000000
         Iterations: 1
         Function evaluations: 13


Multiprocess sampling (2 chains in 2 jobs)
NUTS: [lamb]


JoblibSamplingError: JoblibSamplingError
___________________________________________________________________________
...........................................................................
/Users/stephenhky/anaconda/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/Users/stephenhky/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

............................................

In [ ]:
trace[]